In [39]:
import os
import time
import tensorflow as tf
from tensorflow.keras import models, layers, Model, callbacks
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from process_data import process_data
from nn_gridsearch import logdir

In [2]:
data = process_data()

In [3]:
for k, v in data.items():
    print(k)
    print(type(v))

x_train
<class 'numpy.ndarray'>
x_train_processed
<class 'numpy.ndarray'>
x_test
<class 'numpy.ndarray'>
x_test_processed
<class 'numpy.ndarray'>
y_train
<class 'numpy.ndarray'>
y_test
<class 'numpy.ndarray'>
pipeline
<class 'sklearn.pipeline.Pipeline'>


In [4]:
x_train_processed = data['x_train_processed']
y_train = data['y_train']

In [5]:
x_train_processed.shape

(1047, 11)

> note: old model constrution, using Sequential API

In [6]:
#model = models.Sequential()
#model.add(layers.Dense(32, activation='relu', input_shape=(x_train_processed.shape[1],)))
#model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dense(1, activation='sigmoid'))


> changed to functional API --> more flexibility, for future expansion

In [35]:
input_ = layers.Input(shape=x_train_processed.shape[1:])
hidden_0 = layers.Dense(80, activation='relu')(input_)
hidden_1 = layers.Dense(80, activation='relu')(hidden_0)
hidden_2 = layers.Dense(80, activation='relu')(hidden_1)
hidden_3 = layers.Dense(80, activation='relu')(hidden_2)
output = layers.Dense(1, activation='sigmoid')(hidden_3)
model = Model(inputs=[input_], outputs=[output])

In [36]:
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy', Precision(), Recall(), AUC()])

In [37]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
dense_5 (Dense)              (None, 80)                960       
_________________________________________________________________
dense_6 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_7 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_8 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 81        
Total params: 20,481
Trainable params: 20,481
Non-trainable params: 0
_______________________________________________________

In [41]:
history = model.fit(data['x_train_processed'], data['y_train'], validation_split=.2, epochs=300,
                   callbacks=[
                       EarlyStopping(patience=10, monitor='val_loss', mode='min'),
                       TensorBoard(logdir('custom_model'))])

Epoch 1/300
27/27 [==============================] - 1s 22ms/step - loss: 0.3776 - accuracy: 0.8459 - precision: 0.8275 - recall: 0.7128 - auc: 0.8892 - val_loss: 0.5311 - val_accuracy: 0.7190 - val_precision: 0.6667 - val_recall: 0.6279 - val_auc: 0.7934
Epoch 2/300
27/27 [==============================] - 0s 14ms/step - loss: 0.3802 - accuracy: 0.8530 - precision: 0.8314 - recall: 0.7331 - auc: 0.8813 - val_loss: 0.5848 - val_accuracy: 0.7714 - val_precision: 0.8958 - val_recall: 0.5000 - val_auc: 0.8038
Epoch 3/300
27/27 [==============================] - 0s 10ms/step - loss: 0.3744 - accuracy: 0.8447 - precision: 0.8458 - recall: 0.6858 - auc: 0.8940 - val_loss: 0.5428 - val_accuracy: 0.7905 - val_precision: 0.8000 - val_recall: 0.6512 - val_auc: 0.8236
Epoch 4/300
27/27 [==============================] - 1s 21ms/step - loss: 0.3741 - accuracy: 0.8483 - precision: 0.8477 - recall: 0.6959 - auc: 0.8916 - val_loss: 0.5618 - val_accuracy: 0.7810 - val_precision: 0.8333 - val_recall: 0

In [42]:
%load_ext tensorboard
%tensorboard --logdir ./custom_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4181), started 1:51:55 ago. (Use '!kill 4181' to kill it.)

In [25]:
model = models.load_model('./models/titanic_gridsearch.h5')

In [19]:
model2 = models.load_model('./models/titanic_nn.h5')

In [28]:
results = model.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 0s 18ms/step - loss: 0.5207 - accuracy: 0.7595 - precision_1: 0.8767 - recall_1: 0.5424 - auc_1: 0.8424


In [29]:
results

[0.5206703543663025,
 0.7595419883728027,
 0.8767123222351074,
 0.5423728823661804,
 0.8423963785171509]

In [30]:
model2.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 0s 15ms/step - loss: 0.5051 - accuracy: 0.7672


[0.5050886273384094, 0.767175555229187]

> todo next - function to build model, sklearn wrapper from keras, pipelines, random grid search for model!